In [ ]:
# Extração dos dados

import pandas as pd

df = pd.read_csv('uber_data.csv')

In [ ]:
# Convertendo as datas

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])


In [ ]:
# Tirando valores duplicados

df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Criando a dim_tempo

dim_tempo = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)

dim_tempo['pick_day'] = dim_tempo['tpep_pickup_datetime'].dt.day
dim_tempo['pick_month'] = dim_tempo['tpep_pickup_datetime'].dt.month
dim_tempo['pick_year'] = dim_tempo['tpep_pickup_datetime'].dt.year
dim_tempo['pick_weekday'] = dim_tempo['tpep_pickup_datetime'].dt.weekday
dim_tempo['pick_hour'] = dim_tempo['tpep_pickup_datetime'].dt.hour

dim_tempo['drop_day'] = dim_tempo['tpep_dropoff_datetime'].dt.day
dim_tempo['drop_month'] = dim_tempo['tpep_dropoff_datetime'].dt.month
dim_tempo['drop_year'] = dim_tempo['tpep_dropoff_datetime'].dt.year
dim_tempo['drop_weekday'] = dim_tempo['tpep_dropoff_datetime'].dt.weekday
dim_tempo['drop_hour'] = dim_tempo['tpep_dropoff_datetime'].dt.hour

dim_tempo['datetime_id'] = dim_tempo.index

dim_tempo = dim_tempo[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

In [ ]:
# Criando a dim_tipo_taxa

tipo_taxa = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

dim_tipo_taxa = pd.DataFrame(list(tipo_taxa.items()), columns=['RatecodeID', 'rate_description'])

In [ ]:
# Criando a dim_tipo_pagamento

tipo_pagamento = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

dim_tipo_pagamento = pd.DataFrame(list(tipo_pagamento.items()), columns=['payment_type', 'payment_description'])


In [ ]:
# Criando fato_corridas

fato_corridas = df.merge(dim_tipo_taxa, on = 'RatecodeID') \
                .merge(dim_tipo_pagamento, on = 'payment_type') \
                .merge(dim_tempo, on = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']) \
                [['VendorID', 'datetime_id', 'RatecodeID', 'payment_type', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]